### Project 1 for Programming for Data Analysis. Semester 2 HDip in Data Analytics at ATU, Galway. 
### Author: Shane Keenan 
### Project title: Simulating the economic factors affecting the price of housing (in Ireland) 
---


#### Problem statement (from lecturer Brian McGinley)
For this project you must create a data set by simulating a real-world phenomenon of
your choosing. You may pick any phenomenon you wish – you might pick one that is
of interest to you in your personal or professional life. Then, rather than collect data
related to the phenomenon, you should model and synthesise such data using Python.
We suggest you use the numpy.random package for this purpose.
Specifically, in this project you should:
- Choose a real-world phenomenon that can be measured and for which you could
collect at least one-hundred data points across at least four different variables.
- Investigate the types of variables involved, their likely distributions, and their
relationships with each other.
- Synthesise/simulate a data set as closely matching their properties as possible.
- Detail your research and implement the simulation in a Jupyter notebook – the
data set itself can simply be displayed in an output cell within the notebook.

Note that this project is about simulation – you must synthesise a data set. Some
students may already have some real-world data sets in their own files. It is okay to
base your synthesised data set on these should you wish (please reference it if you do),
but the main task in this project is to create a synthesised data set.

---

- Choose a real-world phenomenon that can be measured and for which you could
collect at least one-hundred data points across at least four different variables.

Choosing a real-world phenomenon is a more difficult task than it first seems. Having considered initially taking on some grande data set such as "*the climate*", i thought better of it and went for something more tangible.
I chose the houseing market. 

The current average monthly house price (*price*) is my primary variable of interest. 

obviously this is quite a multi-variable phenomona, however i wanted to normalise the data set to really only focus on the economic factors. therefore variables such as location, square meterage, number of bedrooms, number of bathrooms, year built and other amenities are not be included and we assume every house is identical and in the same location. 

Variables that I will consider in this project are the current housing supply (*supp*) 



---


- Investigate the types of variables involved, their likely distributions, and their
relationships with each other.





--- 


### References 

factors affecting housing market 

1. https://www.economicshelp.org/blog/377/housing/factors-that-affect-the-housing-market/




#### import packages for python 

In [3]:
import numpy
import pandas 
import matplotlib
import seaborn 